<a href="https://colab.research.google.com/github/bh2005/CMK-Checks/blob/master/what_files_are_needed_to_write_an_API_spezial_age_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To create a custom special agent or datasource for Checkmk that interacts with an API, you'll need a few key files.  The exact number and names might vary slightly depending on your Checkmk version and how you structure your agent, but these are the essentials:

**1. The Agent Script (e.g., `my_api_agent`):**

*   **Location:** `/usr/lib/check_mk_agent/plugins/special/` (or a subdirectory within `special/` for better organization).
*   **Purpose:** This is the core of your agent. It's responsible for:
    *   Fetching data from the API.
    *   Parsing the API response (usually JSON or XML).
    *   Formatting the data into Checkmk's agent output format (`<<<section_name>>>\n...data...\n`).
*   **Language:** Usually Python, but other scripting languages are possible.
*   **Key Elements:**
    *   Import necessary libraries (`requests`, `json`, etc.).
    *   Code to authenticate with the API (if needed).
    *   Functions to make API calls.
    *   Logic to extract the relevant metrics/information.
    *   Formatting of the output using Checkmk's section format.
*   **Example (Simplified Python):**

In [ ]:
#!/usr/bin/env python3

import requests
import json

def main():
    api_url = "https://my-api.com/data"
    try:
        response = requests.get(api_url)
        response.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
        data = response.json()

        print("<<<my_api_data>>>")  # Checkmk section name
        for item in data:
            print(f"metric1={item['value1']} metric2={item['value2']}")

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}") #Important: Print errors to stdout for Checkmk
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
    except Exception as e:
        print(f"Unexpected Error: {e}")

if __name__ == "__main__":
    main()

**2. The Inventory Function (e.g., `my_api_inventory`):**

*   **Location:** `~/local/lib/check_mk/inventory/` (or `/omd/sites/<site>/local/lib/check_mk/inventory/` in OMD).  The filename *must* match the section name from your agent script (without the `<<<>>>`).  So, in this example, it would be `my_api_data`.
*   **Purpose:**  This function is used for *service discovery*. It analyzes the data returned by the agent and determines which services should be monitored.
*   **Language:** Python.
*   **Key Elements:**
    *   Takes the agent output as input.
    *   Parses the agent output (usually line by line).
    *   Uses `yield` to return tuples of `(item_name, parameters)`.  `item_name` is the name of the service Checkmk will create. `parameters` is a dictionary of parameters for the check.
*   **Example (Simplified Python):**

In [ ]:
#!/usr/bin/env python3

import json

def inventory_my_api_data(check_context):
    for line in check_context["my_api_data"]:  # Section name from the agent
        try:
            parts = line.split() #Simple parsing. Adapt as needed.
            item_name = parts[0].split("=")[0] #Example: metric1
            yield (item_name, {}) # No initial parameters in this example
        except Exception as e:
            pass #Handle errors as needed.

**3. The Check Plugin (e.g., `check_my_api_data`):**

*   **Location:** `~/local/lib/check_mk/checks/` (or `/omd/sites/<site>/local/lib/check_mk/checks/` in OMD). The filename *must* match the section name from your agent script (without the `<<<>>>`). So, in this example, it would be `check_my_api_data`.
*   **Purpose:** This script defines how Checkmk will actually *check* the service.  It receives the item name and parameters from the inventory function and the data from the agent.  It then performs the check logic and returns the Checkmk check status (OK, WARNING, CRITICAL, UNKNOWN) and output.
*   **Language:** Python.
*   **Key Elements:**
    *   Takes `item`, `params`, and `check_context` as arguments.
    *   `item` is the item name from the inventory.
    *   `params` are the parameters from the inventory.
    *   `check_context` contains the agent's output (the `<<<my_api_data>>>` section).
    *   Performs the check logic (e.g., comparing values to thresholds).
    *   Returns the Checkmk check status and output.
*   **Example (Simplified Python):**

In [ ]:
#!/usr/bin/env python3

def check_my_api_data(item, params, check_context):
    for line in check_context["my_api_data"]:
        if item in line: #Check the line for the item we are interested in.
            try:
                parts = line.split()
                metric1 = parts[0].split("=")[1]
                # ... (Check logic, e.g., compare metric1 to a threshold) ...
                if int(metric1) > 10:
                    return 2, f"{item} is too high ({metric1})" #CRITICAL
                else:
                    return 0, f"{item} is OK ({metric1})" #OK
            except Exception as e:
                return 3, f"Error processing data: {e}" #UNKNOWN

    return 3, f"Item {item} not found" #UNKNOWN

**4. (Optional) The Wrapper Script (e.g., `agent_my_api_agent`):**

*   **Location:** `~/local/share/check_mk/checks/` (or `/omd/sites/<site>/local/share/check_mk/checks/` in OMD). The filename is `agent_` + the name of your agent.
*   **Purpose:** If you need to pass parameters from your Checkmk rules to your special agent, you need a wrapper script.  This script receives the parameters from Checkmk and then calls your agent script with the appropriate command-line arguments.
*   **Language:** Python.

**Key Considerations:**

*   **Naming:** Consistent naming is crucial.  The section name in the agent, the inventory function name, and the check plugin name should all match.
*   **Error Handling:** Implement robust error handling in all scripts.  Print errors to `stdout` for Checkmk to capture.
*   **Testing:** Test your agent, inventory, and check scripts thoroughly.
*   **Checkmk Version:** Be mindful of compatibility with your Checkmk version.
*   **Packaging (MKP):** For easy distribution and installation, consider creating a Checkmk package (`.mkp`) file.

This breakdown should give you a solid foundation for creating your custom special agent or datasource.  Remember to consult the official Checkmk documentation for the most up-to-date information and best practices. Let me know if you have more questions!